In [74]:
import xlwings as xw
import pandas as pd
import requests
import win32com.client as win32

* [Загрузка данных](#dataset)
* [xlwings](#xlwings)
* [win32com](#win)


# Загрузка данных <a class="anchor" id="daatset"></a>

https://wizard-world-api.herokuapp.com/swagger/index.html

In [78]:
response =  requests.get('https://wizard-world-api.herokuapp.com/Spells')

In [79]:
data = pd.DataFrame(response.json())
data.head()

,id,name,incantation,effect,canBeVerbal,type,light,creator
0,fbd3cb46-c174-4843-a07e-fd83545dce58,Opening Charm,Aberto,Opens doors,True,Charm,Blue,None
1,5eb39a99-72cd-4d40-b4aa-b0f5dd195100,Water-Making Spell,Aguamenti,Conjures water,True,Conjuration,IcyBlue,None
2,ab3a8dfe-1e56-4706-a20d-26afee011ed7,Alarte Ascendare,Alarte Ascendare,Rockets target upward,True,Charm,Red,None
3,c6898622-b914-4609-b3a0-8659ed2ca08a,Animagus Spell,Amato Animo Animato Animagus,Used as part of the ritual to become an Animagus,True,Transfiguration,Gold,None
4,0a11bbf5-702c-45ab-bdd6-20debf00ed39,Anteoculatia,Anteoculatia,Grows antlers on head,True,DarkCharm,Red,None


In [80]:
data = data.drop('id', axis=1)

In [81]:
data['type'].unique()

array(['Charm', 'Conjuration', 'Transfiguration', 'DarkCharm', 'Spell',
       'HealingSpell', 'Jinx', 'Curse', 'MagicalTransportation', 'Hex',
       'CounterSpell', 'DarkArts', 'CounterJinx', 'CounterCharm', 'None',
       'Untransfiguration', 'BindingMagicalContract', 'Vanishment'],
      dtype=object)

# xlwings <a class="anchor" id="xlwings"></a>

https://docs.xlwings.org/en/stable/api/range.html

In [84]:
wb = xw.Book()
sheet = wb.sheets['Лист1']

In [85]:
# переименование листа
sheet.name = 'Несколько таблиц'

In [86]:
# свободный столбец и строчка для записи
# (0, 0) -> A1
# [1, 1] -> A1
free_col = 1
free_row = 1

Для случаев, когда в первой строчке листа есть какая-то запись, и таблица начинается со второй. Если таблица начинается с первой, то ``free_row + 1`` заменяется на ``free_row``.

* первая строка таблицы: 
``sheet.range(free_row + 1, free_col).expand('right')``

* последняя строка таблицы:
``sheet.range(free_row + 1 + len(d), free_col).expand('right')``

* выбрать всю таблицу:
``sheet.range(free_row + 1, free_col).expand()``

In [88]:
for t in data['type'].unique():
    d = data[data['type'] == t].drop(['type'], axis=1)
    
    # кол-во столбцов в таблице
    col_counts = len(d.columns)
    
    # запись в ячейку A1
    sheet[free_row - 1, free_col - 1].value = t
    
    # запись в ячейку A2
    sheet[free_row, free_col - 1].value = d.set_index('name')

    # выделить жирным первую строку 
    sheet.range(free_row, free_col).expand('right').font.bold = True

    # присваивание стиля таблицы 
    sheet.tables.add(sheet.range(free_row + 1, free_col).expand(),
                     table_style_name='TableStyleLight1')

    # изменить цвет фона первой строчки таблицы (заголовки)
    sheet.range(free_row + 1, free_col).expand('right').color = '#7bb1ea' 

    # изменить цвет шрифта первой строчки таблицы (заголовки)
    sheet.range(free_row + 1, free_col).expand('right').font.color = '#ffffff' 

    # изменить цвет фона все таблицы, кроме заголовков
    # можно через expand, но если во второй строчке (от которой начинаем расширение) не заполнен один из столбцов,
    # то выделится только до первого пустого столбца, а не вся строка таблицы
    sheet.range((free_row + 2, free_col), (free_row + 2 + len(d), free_col + col_counts)).color = '#fafafa'  

    # границы таблицы толщиной 2
    sheet.range(free_row + 1, free_col).expand().api.Borders.Weight = 2

    # размер шрифта 12
    sheet.range(free_row + 1, free_col).expand().font.size  = 12

    # ширина столбцов 13
    sheet.range(free_row + 1, free_col).expand().column_width  = 13
    
    # обновляем индекс свободного столбца
    free_col += col_counts + 1

In [89]:
wb.sheets.add('Таблица', after='Несколько таблиц')
sheet2 = wb.sheets['Таблица']

free_col = 1
free_row = 1

In [90]:
# запись в ячейку A1
sheet2[free_row - 1, free_col - 1].options(index=False).value = data

In [91]:
# присваивание стиля таблицы и наименования диапазона
sheet2.tables.add(
                sheet2.range(free_row, free_col).expand(),
                table_style_name='TableStyleLight1',
                name='ИсходныеДанные'
                )

<Table 'ИсходныеДанные' in <Sheet [Книга12]Таблица>>

In [92]:
# автоподбор ширины и высоты
sheet2.autofit()

In [93]:
# теперь построим сводную таблицу с помощью другой библиотеки
# для этого закроем файл, сохранив его с новым названием, а потом откроем с новым
FILE_NAME = 'C:\\Users\\Mosolova_KD\\Downloads\\Пример работы с xlwings.xlsx'
wb.save(FILE_NAME)
wb.close()

Форматирование числовых данных:

``sheet2.range('T:W').api.NumberFormat  = '# ##0'``

``sheet2.range('X:X').api.NumberFormat  = '0,00%'``

# win32com  <a class="anchor" id="win"></a>

https://learn.microsoft.com/ru-ru/office/vba/api/excel.pivottable.pivotfields

Лайфхак: примерную логику для построения сводной таблицы можно понять, включив запись макроса, проделав нужные операции и отсмотрев затем код макроса.

In [97]:
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(FILE_NAME)
win32c = win32.constants

In [98]:
# лист с источником данных
sheet_from = wb.Sheets('Таблица')
# делаем файл видимым
excel.Visible = 1

In [99]:
# создаем новый лист
wb.Sheets.Add(After=wb.Sheets['Таблица']).Name = 'Свод'

In [100]:
# лист, куда вставим сводную таблицу
sheet_to = wb.Sheets('Свод')

In [101]:
#sheet_from.Range("ИсходныеДанные").Select

In [102]:
# создаем сводную таблицу
PivotCache = wb.PivotCaches().Create(SourceType=win32c.xlDatabase,
                                     SourceData='ИсходныеДанные',
                                     Version=win32c.xlPivotTableVersion14)

PivotTable = PivotCache.CreatePivotTable(TableDestination=sheet_to.Range('A1'),
                                         TableName='СводнаяТаблица',
                                         DefaultVersion=win32c.xlPivotTableVersion14)

In [103]:
# заполняем сводную таблицу
# столбец
PivotTable.PivotFields('type').Orientation = win32c.xlRowField
PivotTable.PivotFields('type').Position = 1

In [104]:
# значения: кол-во по полю name, названное 'Кол-во заклинаний'
DataField = PivotTable.AddDataField(
                                    PivotTable.PivotFields('name'),
                                    'Кол-во заклинаний',
                                    win32c.xlCount
                                   )

Функции агрегации: https://learn.microsoft.com/ru-ru/office/vba/api/excel.xlconsolidationfunction

|	Имя	|	Описание	|
|---------------|--------------|
|	xlAverage	|	Среднее	|
|	xlCount	|	Счет	|
|	xlCountNums	|	Подсчитать только числовые значения	|
|	xlDistinctCount	|	Подсчет уникальных значений	|
|	xlMax	|	Максимум	|
|	xlMin	|	Минимум	|
|	xlProduct	|	Умножение	|
|	xlStDev	|	Стандартное отклонение на основе выборки	|
|	xlStDevP	|	Стандартное отклонение на основе всей совокупности	|
|	xlSum	|	Сумма	|
|	xlUnknown	|	Функция промежуточных итогов не указана	|
|	xlVar	|	Дисперсия на основе примера	|
|	xlVarP	|	Дисперсия, основанная на всей совокупности	|


In [106]:
# форматирование числового столбца 
PivotTable.PivotFields('Кол-во заклинаний').NumberFormat = "# ###,0"

In [107]:
# добавление фильтра по полю light
# если нужен еще один фильтр, вместо Position = 1 пишем Position = 2 и т.д.
PivotTable.PivotFields('light').Orientation = win32c.xlPageField
PivotTable.PivotFields('light').Position = 1

In [108]:
# ставим в фильтре по умолчанию Blue, Red и White
for x in data['light'].unique():
    if x in ['Blue', 'Red', 'White']:
        PivotTable.PivotFields('light').PivotItems(x).Visible = True
    else:
        PivotTable.PivotFields('light').PivotItems(x).Visible = False

In [142]:
# очистить фильтры
PivotTable.ClearAllFilters()

In [148]:
# очистить фильтр (выбирает все значения в фильтре)
PivotTable.PivotFields('light').ClearAllFilters()

In [150]:
# перенесем light из фильтров в столбцы
PivotTable.PivotFields('light').Orientation = win32c.xlRowField
PivotTable.PivotFields('light').Position = 2

In [158]:
# показать в табличной форме
PivotTable.RowAxisLayout(win32c.xlTabularRow)

In [160]:
# повторять все подписи элементов 
PivotTable.RepeatAllLabels(win32c.xlRepeatLabels)

In [162]:
# сохраняем
wb.Save()

In [168]:
wb.Close()